In [1]:
#Collect and import ten documents
#I tried to segment this off so it was not pulling such a long part of the destination, but I got errors every other way I sliced it.

import pandas as pd
import os
import numpy as np

documents = []
path = '/Users/smargar/design/text'

filenames=sorted([os.path.join(path, fn) for fn in os.listdir(path)])
print(len(filenames)) # count files in corpus
print(filenames[:10]) # print names of 1st ten files in corpus

13
['/Users/smargar/design/text/American Indian Tales.txt', '/Users/smargar/design/text/British Tales.txt', '/Users/smargar/design/text/German Folklore.txt', '/Users/smargar/design/text/Ireland Tales.txt', '/Users/smargar/design/text/Korean Tales.txt', '/Users/smargar/design/text/Little Puerto Rican Cousin.txt', '/Users/smargar/design/text/Phillipines Tales.txt', '/Users/smargar/design/text/Princess Pocohontas.txt', '/Users/smargar/design/text/Russian Tales.txt', '/Users/smargar/design/text/Scotland Tales.txt']
